In [2]:
from build_dataset import *

In [3]:
from pycocotools.coco import COCO

def build_single_image_dataset(coco_img_dir,
                               instances_json,
                               ref_pickle_file,
                               selected_filenames,
                               use_convert=True):
    # 1. 加载完整数据集
    full_dataset = RefCOCODataset(
        coco_img_dir,
        instances_json,
        ref_pickle_file,
        use_convert=use_convert
    )

    # 2. 用 COCO API 读 JSON，获得所有 img_id 的列表（顺序与 full_dataset.samples 对应）
    coco_api   = COCO(instances_json)
    all_img_ids = coco_api.getImgIds()
    img_infos   = coco_api.loadImgs(all_img_ids)

    # 3. 构建 file_name -> img_id 映射，并打印前 10 个供检查
    fname2id = {info['file_name']: info['id'] for info in img_infos}
    print(">>> JSON 中前 10 个 file_name:", list(fname2id.keys())[:10])

    # 4. 匹配你指定的文件名（支持 exact match 或 endswith）
    selected_ids = []
    for want in selected_filenames:
        matched = False
        for fname, img_id in fname2id.items():
            if fname == want or fname.endswith(want):
                selected_ids.append(img_id)
                matched = True
                break
        if not matched:
            print(f"⚠️ Warning: 在 JSON 中没找到与 '{want}' 对应的 file_name")
    if not selected_ids:
        raise ValueError("❌ 没有任何一个选定文件名匹配成功，先对比上面打印的 file_name 列表吧。")

    # 5. 把对应 img_id 的索引列表算出来
    indices = [i for i, img_id in enumerate(all_img_ids) if img_id in selected_ids]
    if not indices:
        raise ValueError("❌ 虽然匹配到了 img_id，但没能算出对应的索引，请检查 JSON 顺序。")

    # 6. 从 full_dataset.samples 里按索引挑出样本
    try:
        samples_to_save = [full_dataset.samples[i] for i in indices]
    except AttributeError:
        raise AttributeError("❌ full_dataset 上没有 samples 属性，确认 RefCOCODataset 实例里有 samples 列表。")

    # 7. 用一个最小包装把 samples 传给 save_dataset
    class FilteredDataset:
        def __init__(self, samples):
            self.samples = samples

    return FilteredDataset(samples_to_save)


if __name__ == '__main__':
    coco_img_dir    = '/root/data_preprocessing/box_labeled/test_data'
    instances_json  = 'instances.json'
    ref_pickle_file = 'refs(google).p'
    save_path       = 'saved_dataset.json'

    # 这里只写文件名后缀或完整名称都行
    selected = [
        'COCO_train2014_000000098304.jpg',
        '000000098304.jpg',
    ]

    # 得到一个只有你想要那几张图样本的“数据集”
    dataset = build_single_image_dataset(
        coco_img_dir,
        instances_json,
        ref_pickle_file,
        selected_filenames=selected,
        use_convert=True
    )

    # 调用 save_dataset 时就能找到 samples 属性了
    save_dataset(dataset, save_path)
    print(f"✅ 已保存包含 {len(dataset.samples)} 张图的数据集到 {save_path}")

Pickle 文件中第一个 item 的 keys: dict_keys(['sent_ids', 'file_name', 'ann_id', 'ref_id', 'image_id', 'split', 'sentences', 'category_id'])
Raw sentences: [{'tokens': ['zebra', 'creature', 'front', 'and', 'center'], 'raw': 'zebra creature front and center', 'sent_id': 13689, 'sent': 'zebra creature front and center'}, {'tokens': ['zebra'], 'raw': 'zebra', 'sent_id': 13690, 'sent': 'zebra'}, {'tokens': ['whole', 'zebra'], 'raw': 'whole zebra', 'sent_id': 13691, 'sent': 'whole zebra'}] <class 'list'>
loading annotations into memory...
Done (t=3.25s)
creating index...
index created!
>>> JSON 中前 10 个 file_name: ['COCO_train2014_000000098304.jpg', 'COCO_train2014_000000052461.jpg', 'COCO_train2014_000000131074.jpg', 'COCO_train2014_000000524291.jpg', 'COCO_train2014_000000425988.jpg', 'COCO_train2014_000000458762.jpg', 'COCO_train2014_000000194438.jpg', 'COCO_train2014_000000294925.jpg', 'COCO_train2014_000000327694.jpg', 'COCO_train2014_000000060077.jpg']
Dataset 已保存到 saved_dataset.json
Dataset 已